In [1]:
import numpy as np
import pandas as pd
import os
from google.cloud import bigquery
!pip install plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\srish\Documents\NEU\Parallell ML and AI\Untitled Folder\json\My First Project-dd9334d0c0a8.json"

In [3]:
client = bigquery.Client(project='spherical-wave-266309')

ethereum_classic_dataset_ref = client.dataset('crypto_ethereum_classic', project='bigquery-public-data')

In [4]:
query = """
WITH mined_block AS (
  SELECT miner, DATE(timestamp)
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
  WHERE DATE(timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
  ORDER BY miner ASC)
SELECT miner, COUNT(miner) AS total_block_reward 
FROM mined_block 
GROUP BY miner 
ORDER BY total_block_reward DESC
LIMIT 10
"""

query_job = client.query(query)
iterator = query_job.result()

In [5]:
#pip install --upgrade --user google-cloud-bigquery

In [6]:
rows = list(iterator)
#Transform rows in pandas dataframe
top_miners = pd.DataFrame(data=[list(x.values()) for x in rows],columns = list(rows[0].keys()))
#top 10
top_miners.head(10)

,miner,total_block_reward
0,0xdf7d7e053933b5cc24372f878c90e62dadad5d42,63532
1,0x594a6de78a147371e8288c3a09fa94d673df63e1,36516
2,0x9eab4b0fc468a7f5d46228bf5a76cb52370d068d,19979
3,0x0073cf1b9230cf3ee8cab1971b8dbef21ea7b595,13267
4,0x6a609b873a02369dbffe6dc287a20786bdb1a029,9561
5,0x004730417cd2b1d19f6be2679906ded4fa8a64e2,5591
6,0x04cb5f569baaa5b0f6b2b07b910e6fefbc597f73,3085
7,0x7d0ae1b43b5e3499bb03f1d66ecec1affb0dc70e,2754
8,0x60f814acce2b2129707228c61065696bcc3e8b9f,2716
9,0x1db733f5359413e06b5dcf85d137b61ad7f3e3b3,2553


In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [12]:
labels = top_miners['miner']
values = top_miners['total_block_reward']
trace = go.Pie(labels = labels, values = values)
iplot([trace])

In [13]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

SELECT miner, 
    DATE(timestamp) as date,
    COUNT(miner) as total_block_reward
FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
GROUP BY miner, date
HAVING COUNT(miner) > 100
ORDER BY date, COUNT(miner) ASC
"""
query_job = client.query(query)
iterator = query_job.result()

In [14]:
rows = list(iterator)
#Transform rows in pandas dataframe
top_miners_by_date = pd.DataFrame(data=[list(x.values()) for x in rows],columns = list(rows[0].keys()))
#top 10
top_miners_by_date.head(10)

,miner,date,total_block_reward
0,0x9beb4d346c6309a9ce4f27393c6322c8f03a3be6,2015-07-30,104
1,0xa1623430350c5df1b52b0b57483a5bb45d1796da,2015-07-30,130
2,0x0193d941b50d91be6567c7ee1c0fe7af498b4137,2015-07-30,147
3,0xbb12b5a9b85d4ab8cde6056e9c1b2a4a337d2261,2015-07-30,197
4,0xd7e30ae310c1d1800f5b641baa7af95b2e1fd98c,2015-07-30,205
5,0xf927a40c8b7f6e07c5af7fa2155b4864a4112b13,2015-07-30,309
6,0x1b7047b4338acf65be94c1a3e8c5c9338ad7d67c,2015-07-30,453
7,0x28921e4e2c9d84f4c0f0c0ceb991f45751a0fe93,2015-07-30,464
8,0x9dfc0377058b7b9eb277421769b56df1395705f0,2015-07-30,489
9,0xbb7b8287f3f0a933474a79eae42cbca977791171,2015-07-30,1745


In [15]:
date_series = top_miners_by_date['date'].unique()
date_series

array([datetime.date(2015, 7, 30), datetime.date(2015, 7, 31),
       datetime.date(2015, 8, 1), ..., datetime.date(2020, 7, 19),
       datetime.date(2020, 7, 20), datetime.date(2020, 7, 21)],
      dtype=object)

In [19]:
traces = []
miner_series = top_miners_by_date['miner'].unique()

In [17]:
for index, miner in enumerate(miner_series):
    miner_reward_by_date = top_miners_by_date.loc[top_miners_by_date['miner'] == miner]
    miner_reward = miner_reward_by_date['total_block_reward']
    miner_date = miner_reward_by_date['date']
    trace = dict(
        x=miner_date,
        y=miner_reward,
        mode='lines',
        stackgroup='one'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig)

In [20]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

WITH total_reward_book AS (
  SELECT miner, 
    DATE(timestamp) as date,
    COUNT(miner) as total_block_reward
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
  GROUP BY miner, date
  HAVING COUNT(miner) > 100
),
total_reward_book_by_date AS (
 SELECT date, 
        miner AS address, 
        SUM(total_block_reward / POWER(10,0)) AS value
  FROM total_reward_book
  GROUP BY miner, date
),
daily_rewards_with_gaps AS (
  SELECT
    address, 
    date,
    SUM(value) OVER (PARTITION BY ADDRESS ORDER BY date) AS block_rewards,
    LEAD(date, 1, CURRENT_DATE()) OVER (PARTITION BY ADDRESS ORDER BY date) AS next_date
  FROM total_reward_book_by_date
),
calendar AS (
  SELECT date 
  FROM UNNEST(GENERATE_DATE_ARRAY('2015-07-30', CURRENT_DATE())) AS date
),
daily_rewards AS (
  SELECT address, 
    calendar.date, 
    block_rewards
  FROM daily_rewards_with_gaps
  JOIN calendar ON daily_rewards_with_gaps.date <= calendar.date 
  AND calendar.date < daily_rewards_with_gaps.next_date
),
supply AS (
  SELECT date,
    SUM(block_rewards) AS total_rewards
  FROM daily_rewards
  GROUP BY date
),
ranked_daily_rewards AS (
  SELECT daily_rewards.date AS date,
    block_rewards,
    ROW_NUMBER() OVER (PARTITION BY daily_rewards.date ORDER BY block_rewards DESC) AS rank
  FROM daily_rewards
  JOIN supply ON daily_rewards.date = supply.date
  WHERE SAFE_DIVIDE(block_rewards, total_rewards) >= 0.01
  ORDER BY block_rewards DESC
),
daily_gini AS (
  SELECT date,
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * SUM((block_rewards * (rank - 1) + block_rewards / 2)) / COUNT(*) / SUM(block_rewards) AS gini
  FROM ranked_daily_rewards
  GROUP BY DATE
)
SELECT date,
  gini,
  AVG(gini) OVER (ORDER BY date ASC ROWS 7 PRECEDING) AS gini_sma_7,
  AVG(gini) OVER (ORDER BY date ASC ROWS 30 PRECEDING) AS gini_sma_30
FROM daily_gini
ORDER BY date ASC
"""

query_job = client.query(query)
iterator = query_job.result()

In [21]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
mining_reward_gini_by_date = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
mining_reward_gini_by_date.head(10)

,date,gini,gini_sma_7,gini_sma_30
0,2015-07-30,0.465213,0.465213,0.465213
1,2015-07-31,0.524871,0.495042,0.495042
2,2015-08-01,0.586563,0.525549,0.525549
3,2015-08-02,0.576192,0.538210,0.538210
4,2015-08-03,0.514817,0.533531,0.533531
5,2015-08-04,0.514918,0.530429,0.530429
6,2015-08-05,0.497703,0.525754,0.525754
7,2015-08-06,0.520803,0.525135,0.525135
8,2015-08-07,0.511116,0.530873,0.523577
9,2015-08-08,0.502773,0.528111,0.521497


In [22]:
traces = []
x = mining_reward_gini_by_date['date']
gini_list = ['gini', 'gini_sma_7', 'gini_sma_30']
for gini in gini_list:
    y = mining_reward_gini_by_date[gini]
    trace = dict(
        x=x,
        y=y,
        hoverinfo=f'{gini}',
        mode='lines'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig, validate=False)

In [23]:
query = """
with double_entry_book as (
    -- debits
    select to_address as address, value as value
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select miner as address, sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum_classic.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum_classic.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner
    union all
    -- transaction fees credits
    select from_address as address, -(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum_classic.transactions`
)
select address, 
sum(value) / 1000000000 as balance
from double_entry_book
group by address
order by balance desc
limit 20
"""

query_job = client.query(query)
iterator = query_job.result()

In [24]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
top_address_rich_list = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
top_address_rich_list.head(10)

,address,balance
0,0x149e8a848113704d0cf8c24060fdc60867680119,6119502691199000.000245
1,0x8237a5caf43c8066ce4a2faca644d23b238a372c,5378466902047692.209622897
2,0x71d661e9dcd6cc8dd67de5ac6f8556ac68a701d3,4478381569776070
3,0xc5711c304b88233be31a261c540f01398a77181f,4454579888458640.1
4,0x2ffdb9ac046f21a105d344a98754f5e438c84511,3535561256376700
5,0x59af2dda5c57b8f17f2714bfe41232fe17a2482c,3412273651124195
6,0x5e8f0e63e7614c47079a41ad4c37be7def06df5a,3360332283233543.895444629
7,0x78d5e220b4cc84f290fae4148831b371a851a114,2968326266305841.556717828
8,0x8074a5c78fd9c3b2e90012a83528aa3c313a98ea,2293332099677560.1
9,0x6c480413ee251cb0df9606d1cb939859f85b0a93,2120000007060000


In [25]:
labels = top_address_rich_list['address']
values = top_address_rich_list['balance']

trace = go.Pie(labels=labels, values=values)

iplot([trace])

In [26]:
query = """
with 
double_entry_book as (
    -- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select miner as address, sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum_classic.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner, block_timestamp
    union all
    -- transaction fees credits
    select from_address as address, -(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.transactions`
),
double_entry_book_by_date as (
    select 
        date(block_timestamp) as date, 
        address, 
        sum(value / POWER(10,0)) as value
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select 
        address, 
        date,
        sum(value) over (partition by address order by date) as balance,
        lead(date, 1, current_date()) over (partition by address order by date) as next_date
        from double_entry_book_by_date
),
calendar as (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
),
 supply as (
    select
        date,
        sum(balance) as daily_supply
    from daily_balances
    group by date
),
ranked_daily_balances as (
    select 
        daily_balances.date,
        balance,
        row_number() over (partition by daily_balances.date order by balance desc) as rank
    from daily_balances
    join supply on daily_balances.date = supply.date
    where safe_divide(balance, daily_supply) >= 0.0001
    ORDER BY safe_divide(balance, daily_supply) DESC
), 
gini_daily as (
   select
    date,
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * sum((balance * (rank - 1) + balance / 2)) / count(*) / sum(balance) as gini
  from ranked_daily_balances
  group by date
)
select date,
    gini,
    avg(gini) over (order by date asc rows 7 preceding) as gini_sma7,
    avg(gini) over (order by date asc rows 30 preceding) as gini_sma30
from gini_daily
order by date asc
"""

query_job = client.query(query)
iterator = query_job.result()



In [27]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
daily_balance_gini = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
daily_balance_gini.head(10)

,date,gini,gini_sma7,gini_sma30
0,2015-07-30,0.689518,0.689518,0.689518
1,2015-07-31,0.689473,0.689495,0.689495
2,2015-08-01,0.689358,0.689449,0.689449
3,2015-08-02,0.689275,0.689406,0.689406
4,2015-08-03,0.689175,0.689360,0.689360
5,2015-08-04,0.689103,0.689317,0.689317
6,2015-08-05,0.689097,0.689285,0.689285
7,2015-08-06,0.689067,0.689258,0.689258
8,2015-08-07,0.697556,0.690263,0.690180
9,2015-08-08,0.702019,0.691831,0.691364


In [28]:
traces = []
x = daily_balance_gini['date']
gini_list = ['gini', 'gini_sma7', 'gini_sma30']
for gini in gini_list:
    y = daily_balance_gini[gini]
    trace = dict(
        x=x,
        y=y,
        hoverinfo=f'{gini}',
        mode='lines'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig, validate=False)

In [29]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

WITH block_rows AS (
  SELECT *, ROW_NUMBER() OVER (ORDER BY timestamp) AS rn
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks`
),
delta_time AS (
  SELECT
  mp.timestamp AS block_time,
  mp.difficulty AS difficulty,
  TIMESTAMP_DIFF(mp.timestamp, mc.timestamp, SECOND) AS delta_block_time
  FROM block_rows mc
  JOIN block_rows mp
  ON mc.rn = mp.rn - 1
),
hashrate_book AS (
  SELECT TIMESTAMP_TRUNC(block_time, DAY) AS block_day,
  AVG(delta_block_time) as daily_avg_block_time,
  AVG(difficulty) as daily_avg_difficulty
  FROM delta_time
  GROUP BY TIMESTAMP_TRUNC(block_time, DAY)
)
SELECT block_day,
(daily_avg_difficulty/daily_avg_block_time)/1000000000 as hashrate
FROM hashrate_book
ORDER BY block_day ASC
"""

query_job = client.query(query)
iterator = query_job.result()

In [31]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
daily_hashrate = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
daily_hashrate.head(10)

,block_day,hashrate
0,2015-07-30 00:00:00+00:00,0.000583
1,2015-07-31 00:00:00+00:00,47.939455
2,2015-08-01 00:00:00+00:00,54.303050
3,2015-08-02 00:00:00+00:00,63.291144
4,2015-08-03 00:00:00+00:00,68.788433
5,2015-08-04 00:00:00+00:00,75.580872
6,2015-08-05 00:00:00+00:00,80.448478
7,2015-08-06 00:00:00+00:00,81.771576
8,2015-08-07 00:00:00+00:00,88.339527
9,2015-08-08 00:00:00+00:00,96.474665


In [33]:
 trace = go.Scatter(
    x=daily_hashrate['block_day'],
    y=daily_hashrate['hashrate'],
    mode='lines'
)
data = [trace]
iplot(data)